# EEG Preprocessing using independent component analysis (ICA)
ICA can be used for artefact detection, since it identifies seperate components of the signal that have been combined during recording. That means that we can actually separate noise compoenents, such as eye blinks, from the rest of the signal, and thereby exclude them.

**Notes:**
* figure out how to exclude practise trials
* add trigger for mouse-click
* add trigger for maximal deaccelaration in incongruent trials
* remove eog when doing ICA???
* remove data after experiment finished?
    * determine length of experiment from behavioural data
* Do we filter before doing ICA - yes according to first link below! 


NICE LINKS:https://erpinfo.org/blog/2018/6/18/hints-for-using-ica-for-artifact-correction & https://labeling.ucsd.edu/tutorial/labels

## Loading modules & data

In [ ]:
# importing modules
import numpy as np
import mne
#! pip install mne

In [ ]:
raw = mne.io.read_raw_brainvision('Stroop_mouse_EEG_data/EEG/Group7_own.vhdr', eog=('EOG1', 'EOG2'), preload = True)

In [ ]:
# Removing EEG data which was recorded when the experiment had ended
raw.crop(tmin=0.0, tmax=420, include_tmax=True)

## Specifing the channel locations using the montage-related functions

In [ ]:
montage = mne.channels.make_standard_montage('standard_1020') 
raw.set_montage(montage, verbose=False)

## Redefine the reference to a common average

In [ ]:
raw.set_eeg_reference('average', projection=False, verbose=False)

## Preparing data for ICA
High-pass filtering the data at 0.1 Hz and subsequently low-pass filtering at 40 Hz

In [ ]:
raw = raw.filter(0.1, None)
raw = raw.filter(None, 40)

## Setting up and fitting the ICA
Fitting the ICA with 800 iterations with a random seed at 97. n_components=0.95 ensures that the number of components selected explain at least 95% of the variance in the data

In [ ]:
ica = mne.preprocessing.ICA(n_components=0.95, random_state=97, max_iter=800)
ica.fit(raw)

## Plotting of ICA
### Plotting of components

In [ ]:
ica.plot_components();

### Plotting of the time series of the ICA

In [ ]:
ica.plot_sources(raw, show_scrollbars=False);

## Exclusion of components
From visual inspection of the topographic maps and the time series plots of the components, we can see that the ____  component seems to capture noise especially from eyeblinks. Consequently, we can remove it and thereby extract these artefacts from the data.a


In [ ]:
# the first component is excluded based on visual inspection
ica.exclude = [1]
ica.plot_properties(raw, picks=ica.exclude);

## Applying ICA to the data

In [ ]:
ica.apply(raw)

In [ ]:
# plotting the data after filtering and ica
raw.plot(n_channels = 33, scalings = {'eeg': 100e-6});

### Epoching the data

In [ ]:
events, _  = mne.events_from_annotations(raw)

In [ ]:
# Determining all the unique triggers
np.unique(events[:,2])

In [ ]:
# Creating a dictionary with event ids
event_id = {'Image/cNeu': 11, # Image trigger neutral condition 
            'Image/cCon': 21, # Image trigger congruent condition  
            'Image/cInc': 31, # Image trigger incongruent condition 
            'Word/cNeu': 12, # Word trigger neutral condition
            'Word/cCon': 22, # Word trigger congruent condition
            'Word/cInc': 32 # Word trigger incongruent condition
} 

In [ ]:
mne.viz.plot_events(events, first_samp=raw.first_samp, event_id=event_id);

#### Creating Epochs
**Note:** Determine sensible time window

In [ ]:
# establishing time window
tmin, tmax = -0.2, 0.8

In [ ]:
# rejecting all epochs with values exeeding 150 micro volts - cannot be brain data
reject = {'eeg': 150e-6}

In [ ]:
# choosing only EEG channels for epoching
picks = mne.pick_types(raw.info, eeg=True, eog=False)

In [ ]:
# creating the epochs using the variables created in the cell above, and timelocking to the events
# baseline time interval spans from beginning of the data (-0.2 s) to 0 s (stimulus onset)
# we use the reject variable we created earlier in order to remove artefacts
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks,
                    baseline=(None, 0), reject=reject, preload=True, verbose = True)

In [ ]:
# Downsampling to 250 Hz
epochs_resampled = epochs.resample(250)

### Plot of consistency of participant’s brain responses as measured at Oz to all the words and to all the images

In [ ]:
img_epochs = epochs_resampled['Image']
word_epochs = epochs_resampled['Word']

In [ ]:
# plotting one channel as an example for each modality
# now the '/' used in the event IDs comes in handy! To get all epochs with image stimuli, we can index 'image' across the left/right dimension 
img_epochs.plot_image(picks=['Oz']);
word_epochs.plot_image(picks=['Oz']);

### Creating evoked variables

In [ ]:
image_evoked = img_epochs.average()
word_evoked = word_epochs.average()

In [ ]:
mne.viz.plot_compare_evokeds({'image': image_evoked, 'word': word_evoked},
                             legend='upper left', show_sensors='upper right')

### Eliciting response times using the triggers from the button-presses: Copy the code from the rt_code_snippet.py in the scripts-folder to your notebook and try to run it (it’s a bit dense, I know). Can you get a sensible output? If not, we’ll play around with this at a later stage.

In [ ]:
image_evoked.plot_joint(picks='eeg');

In [ ]:
word_evoked.plot_joint(picks='eeg');

As a final note, we can create contrasts (aka. difference waves) on the fly by using the combine_evoked-function (in combination with the weights=[1, -1]-parameter).

This also gives us a chance to try out one final plotting function, namely the plot_topo-function. This function plots the difference waves (or whichever traces we choose to plot) for each channel in their positions on the scalp.

In [ ]:
evoked_diff = mne.combine_evoked([word_evoked, image_evoked], weights=[1, -1])
evoked_diff.pick_types(eeg=True).plot_topo(color='r', legend=False)